catatan awal:

> untuk file .h5
*     header untuk data 'X' : "x_images"
*     header untuk data 'y' : "y_labels"

> kode label
0. ceplok
1. kawung
2. lereng
3. nitik
4. parang
5. semen
6. lung-lungan

In [5]:
import h5py
import numpy as np
from PIL import Image
import os
from sklearn.utils import shuffle
import imageio
import cv2 # for zooming in image
import math
import random
from random import randint # will be used in random_erasing_image function
from scipy.ndimage import zoom, rotate

# METHODS

In [6]:
def slice_image(image):
    """ Slice image into 4 parts
        Parameters:
        ---------------
        image            array of RGB (tuple), numpy array of image 
 
        Returns:
        ----------
        images_array     array of images sized 4 
    """
    height, width = image.shape[:2]
    center_h = int(np.round(height/2))
    center_w = int(np.round(width/2))
    
    new_images = []
    pos_w = 0
    pos_h = 0
    for _ in range(4):
        if pos_w <= width:
            new_images.append(image[pos_h:pos_h+center_h, pos_w:pos_w+center_w])
            pos_w = pos_w+center_w
            if pos_w >= width:
                pos_w = 0
                pos_h = pos_h+center_h
        elif pos_w > width and pos_h < height:
            new_images.append(image[pos_h:pos_h+center_h, pos_w:pos_w+center_w])
    return new_images

In [9]:
img = cv2.imread(f"testes.jpg")
                
# convert to grayscale 3 channels
# img = cv2.cvtColor(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY), cv2.COLOR_GRAY2RGB)

# slicing Image into 4
img_for_slice = cv2.resize(img, (450,450))    
new = slice_image(img_for_slice)                
for i in range(len(new)): 
    # save 4 sliced images part
    cv2.imwrite(f"tes{i}.jpg", new[i])

In [18]:
def zoom_in_image(img, min_zoom=1.1, max_zoom=1.8):
    """
    Center zoom in/out of the given image and returning an enlarged/shrinked view of 
    the image without changing dimensions
    Args:
        img : Image array
        zoom_factor : amount of zoom as a ratio (0 to Inf)
    """
    zoom_factor = random.uniform(min_zoom, max_zoom)
    zoom_factor = round(zoom_factor,1)
    
    height, width = img.shape[:2] # It's also the final desired shape
    new_height, new_width = int(height * zoom_factor), int(width * zoom_factor)

    ### Crop only the part that will remain in the result (more efficient)
    # Centered bbox of the final desired size in resized (larger/smaller) image coordinates
    y1, x1 = max(0, new_height - height) // 2, max(0, new_width - width) // 2
    y2, x2 = y1 + height, x1 + width
    bbox = np.array([y1,x1,y2,x2])
    # Map back to original image coordinates
    bbox = (bbox / zoom_factor).astype(np.int)
    y1, x1, y2, x2 = bbox
    cropped_img = img[y1:y2, x1:x2]

    # Handle padding when downscaling
    resize_height, resize_width = min(new_height, height), min(new_width, width)
    pad_height1, pad_width1 = (height - resize_height) // 2, (width - resize_width) //2
    pad_height2, pad_width2 = (height - resize_height) - pad_height1, (width - resize_width) - pad_width1
    pad_spec = [(pad_height1, pad_height2), (pad_width1, pad_width2)] + [(0,0)] * (img.ndim - 2)

    result = cv2.resize(cropped_img, (resize_width, resize_height))
    result = np.pad(result, pad_spec, mode='constant')
    assert result.shape[0] == height and result.shape[1] == width
    return result

def rotate_image(img):
    """
    generate new rotated image

    Arguments:\n
    img --> Array, pixels array of image\n
    degree --> float, rotation degree within range -150 to 150\n
    """
    degree_value = randint(-225, 225)
    out = rotate(img, degree_value, mode='mirror')
    return out

def flip_ud_image(img):
    """
    generate flipped image on x axis\n
    """
    out = np.flipud(img)
    return out

def random_erasing_image(img, min_percent=20, max_percent=30):
    """
    generate new random erased image.\n
    random erasing area is 20%-30% of image\n
    """
    #
    #first:
    #generate random number for random erasing area percentage
    # (default is 20% to 30%)
    #
    percent_number = randint(min_percent,max_percent)/100

    #
    #second:
    #generate random number for random erasing area position (x,y)
    #
    h, w = img.shape[:2]
    max_length_h = math.floor(h*percent_number)
    max_length_w = math.floor(w*percent_number)
    # pos_h and pos_w set, so not to close to edge
    pos_h = randint(0,h)
    pos_w = randint(0,w)
    end_pos_h = pos_h + max_length_h
    end_pos_w = pos_w + max_length_w

    #
    #third:
    #generate black area
    #
    black = np.zeros_like(img)
    black = black[pos_h:end_pos_h, pos_w:end_pos_w]

    #
    #final:
    #create the new image
    #
    out = img
    out[pos_h:end_pos_h, pos_w:end_pos_w] = black
    return out

def mix_augmentation(img,rand_value):
    value = rand_value
    if value == 1:
        out = zoom_in_image(img)
        out = random_erasing_image(out) 
    elif value == 2:
        out = rotate_image(img)
        out = flip_ud_image(out)
    elif value == 3:
        out = flip_ud_image(img)
        out = random_erasing_image(out)
    elif value == 4:
        out = rotate_image(img)
        out = zoom_in_image(out)
    elif value == 5:
        out = rotate_image(img)
        out = random_erasing_image(out)
    elif value == 6:
        out = flip_ud_image(img)
        out = zoom_in_image(out)
        
    return out, out.shape[0], out.shape[1]

def do_mix(img, rand_value):
    out, h, w = mix_augmentation(img, rand_value)
    while h == 1 and w == 1:
        out, h, w = mix_augmentation(img, rand_value)
    
    return out

In [17]:
def create_augmented_dataset_allfeatures(folder_dir, dest_dir):    
    """
    produce 4 sliced images, 4 Augmented images, and 1 mix-augment image
    """
    try:
        os.makedirs(dest_dir)
    except OSError as error: 
        print(error)
        
    new_dir = dest_dir    
    for folder_image in os.listdir(folder_dir): 
        try: 
            os.mkdir(f"{new_dir+folder_image}") 
        except OSError as error: 
            print(error)
        for filename in os.listdir(f"{folder_dir+folder_image}"):
            if filename.endswith(".png") or filename.endswith(".jpg") or filename.endswith(".PNG") or filename.endswith(".JPG") or filename.endswith(".jpeg") or filename.endswith(".JPEG"):
                print(f"working on {folder_image}\\{filename}")

                # read image in RGB
                img = cv2.imread(f"{folder_dir+folder_image}\\{filename}", 0)
                img = cv2.cvtColor(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY), cv2.COLOR_GRAY2RGB)
                
                # resize to 450x450
                img_for_slice = cv2.resize(img.copy(), (450,450))
                # slice image into 4 parts
                slices = slice_image(img_for_slice)
                for i in range(len(slices)):
                    cv2.imwrite(f"{new_dir+folder_image}\\SL%{str(i)}%=={filename}", slices[i])
                
                # create the MI
                val = randint(1,6)
                mixed_augment_image = do_mix(img.copy(), val)
                cv2.imwrite(f"{new_dir+folder_image}\\SL%MI%=={filename}", mixed_augment_image)
                             
                # create the RA
                cv2.imwrite(f"{new_dir+folder_image}\\SL%RA=={filename}", random_erasing_image(img.copy()))
                # create the FL
                cv2.imwrite(f"{new_dir+folder_image}\\SL%FL=={filename}", flip_ud_image(img.copy()))
                # create the ZO
                cv2.imwrite(f"{new_dir+folder_image}\\SL%ZO=={filename}", zoom_in_image(img.copy()))
                # create the RO
                cv2.imwrite(f"{new_dir+folder_image}\\SL%RO=={filename}", rotate_image(img.copy()))
                    
            else:
                print(f"FOUND OTHER TYPE: {filename}")

In [19]:
def create_augmented_nongeometri_for_GAN(folder_dir, dest_dir):        
    new_dir = dest_dir
    for folder_image in os.listdir(folder_dir): 
        try: 
            os.mkdir(f"{new_dir+folder_image}") 
        except OSError as error: 
            print(error)
        for filename in os.listdir(f"{folder_dir+folder_image}"):
            if filename.endswith(".png") or filename.endswith(".jpg") or filename.endswith(".PNG") or filename.endswith(".JPG") or filename.endswith(".jpeg") or filename.endswith(".JPEG"):
                print(f"working on {folder_image}\\{filename}")

                # copy the original image to img_others and img_for_random_erase
                # img_others is used by zoomed, flipped, rotated
                # img_for_random_erase is used by random erasing
                # and img_for_mix used by mixed augmentation
                img_others = cv2.imread(f"{folder_dir+folder_image}\\{filename}")
                img2 = img_others.copy()
                img3 = img_others.copy()

                # FIRST
                # add original image to new directory
                cv2.imwrite(f"{new_dir+folder_image}\\{filename}", img_others)

                # SECOND
                # create image for other augmentation methods
                flipped_image = flip_ud_image(img_others.copy())
                rotated_image = rotate_image(img_others.copy())
                status = cv2.imwrite(f"{new_dir+folder_image}\\FL=={filename}", flipped_image)
                status = cv2.imwrite(f"{new_dir+folder_image}\\RO=={filename}", rotated_image)

                # THIRD
                # slicing Image into 4
                img = cv2.imread(f"{folder_dir+folder_image}\\{filename}")
                if np.shape(img) < (448,448,):
                    img_for_slice = cv2.resize(img, (450,450))
                else:
                    img_for_slice = img.copy()
                img_for_slice = np.array(img)
                new = slice_image(img)                
                for i in range(len(new)): 
                    # save 4 sliced images part
                    cv2.imwrite(f"{new_dir+folder_image}\\SL%{str(i)}%=={filename}", new[i])
                
            else:
                print(f"FOUND OTHER TYPE: {filename}")

In [23]:
def create_dataset_noaugm(folder_dir, dest_dir):      
    """
    Produce 4 sliced images with no augmentation
    """
    try:
        os.makedirs(dest_dir)
    except OSError as error: 
        print(error)
        
    new_dir = dest_dir
    
    for folder_image in os.listdir(folder_dir):    
        try: 
            os.mkdir(f"{new_dir+folder_image}") 
        except OSError as error: 
            print(error)
        for filename in os.listdir(f"{folder_dir+folder_image}"):
            if filename.endswith(".png") or filename.endswith(".jpg") or filename.endswith(".PNG") or filename.endswith(".JPG") or filename.endswith(".jpeg") or filename.endswith(".JPEG"):
                print(f"working on {folder_image}\\{filename}")
                img = cv2.imread(f"{folder_dir+folder_image}\\{filename}")
                
                # convert to grayscale 3 channels
                # img = cv2.cvtColor(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY), cv2.COLOR_GRAY2RGB)
                
                # slicing Image into 4
                img_for_slice = cv2.resize(img, (450,450))    
                new = slice_image(img_for_slice)                
                for i in range(len(new)): 
                    # save 4 sliced images part
                    cv2.imwrite(f"{new_dir+folder_image}\\SL%{str(i)}%=={filename}", new[i])
            else:
                print(f"FOUND OTHER TYPE: {filename}")

In [21]:
def create_dataset_singleaugm(method, folder_dir, dest_dir):     
    """
    Produce 4 sliced images, and do augmentation to 1 part randomly
    """
    try:
        os.makedirs(dest_dir)
    except OSError as error: 
        print(error)
    new_dir = dest_dir
    method = method.lower() # make insensitive case by bringing to lowercase
    for folder_image in os.listdir(folder_dir):    
        try: 
            os.mkdir(f"{new_dir+folder_image}") 
        except OSError as error: 
            print(error)
        for filename in os.listdir(f"{folder_dir+folder_image}"):
            if filename.endswith(".png") or filename.endswith(".jpg") or filename.endswith(".PNG") or filename.endswith(".JPG") or filename.endswith(".jpeg") or filename.endswith(".JPEG"):
                print(f"working on {folder_image}\\{filename}")
                
                img = cv2.imread(f"{folder_dir+folder_image}\\{filename}")
                img_for_slice = cv2.resize(img, (450,450))  
                
                # rand int 0 to 3 to choose which part to be augmented
                part_idx = randint(0,3)                
                
                # swith case which method to be executed 
                if method == 'ra':                    
                    # augment the sliced part
                    new = slice_image(img_for_slice)
                    for i in range(len(new)): 
                        if i == part_idx:
                            cv2.imwrite(f"{new_dir+folder_image}\\{str(i)}=={filename}", new[i])
                            cv2.imwrite(f"{new_dir+folder_image}\\RA%{str(i)}=={filename}", random_erasing_image(new[i].copy()))
                        else:
                            cv2.imwrite(f"{new_dir+folder_image}\\{str(i)}=={filename}", new[i])
                        
                elif method == 'ro':                    
                    # augment the sliced part
                    new = slice_image(img_for_slice)
                    for i in range(len(new)): 
                        if i == part_idx:
                            cv2.imwrite(f"{new_dir+folder_image}\\{str(i)}=={filename}", new[i])
                            cv2.imwrite(f"{new_dir+folder_image}\\RO%{str(i)}=={filename}", rotate_image(new[i].copy()))
                        else:
                            cv2.imwrite(f"{new_dir+folder_image}\\{str(i)}=={filename}", new[i])
                            
                elif method == 'fl':                    
                    # augment the sliced part
                    new = slice_image(img_for_slice)
                    for i in range(len(new)): 
                        if i == part_idx:
                            cv2.imwrite(f"{new_dir+folder_image}\\{str(i)}=={filename}", new[i])
                            cv2.imwrite(f"{new_dir+folder_image}\\FL%{str(i)}=={filename}", flip_ud_image(new[i].copy()))
                        else:
                            cv2.imwrite(f"{new_dir+folder_image}\\{str(i)}=={filename}", new[i])
                    
                elif method == 'zo':                    
                    # augment the sliced part
                    new = slice_image(img_for_slice)
                    for i in range(len(new)): 
                        if i == part_idx:
                            cv2.imwrite(f"{new_dir+folder_image}\\{str(i)}=={filename}", new[i])
                            cv2.imwrite(f"{new_dir+folder_image}\\ZO%{str(i)}=={filename}", zoom_in_image(new[i].copy()))
                        else:
                            cv2.imwrite(f"{new_dir+folder_image}\\{str(i)}=={filename}", new[i])
                            
                elif method == 'mi':                    
                    # augment the sliced part
                    new = slice_image(img_for_slice)
                    for i in range(len(new)): 
                        if i == part_idx:
                            # set up to create 'mixed augmentation' image
                            val = randint(1,6)
                            mixed_augment_image = do_mix(new[i].copy(), val)
                            cv2.imwrite(f"{new_dir+folder_image}\\MI%{str(i)}=={filename}", zoom_in_image(mixed_augment_image))
                            # save the original one as well        
                            cv2.imwrite(f"{new_dir+folder_image}\\{str(i)}=={filename}", new[i])
                        else:
                            cv2.imwrite(f"{new_dir+folder_image}\\{str(i)}=={filename}", new[i])
                    
            else:
                print(f"FOUND OTHER TYPE: {filename}")
                


### MAIN CELL BELOW

In [10]:
source_train = "C:\\SAMUEL\\KULIAH\\Skripsi Gasal 1920\\jupyter-notebook\\dataset_batik\\nongeometri\\training\\"
dest_train = f"C:\\SAMUEL\\KULIAH\\Skripsi Gasal 1920\\jupyter-notebook\\new\\lunglungan_gan\\"
    
create_dataset_noaugm(source_train, dest_train)

[WinError 183] Cannot create a file when that file already exists: 'C:\\SAMUEL\\KULIAH\\Skripsi Gasal 1920\\jupyter-notebook\\new\\lunglungan_gan\\'
working on 5\10.PNG
working on 5\100.PNG
working on 5\102.PNG
working on 5\1023.PNG
working on 5\109.PNG
working on 5\11.PNG
working on 5\111.PNG
working on 5\122.PNG
working on 5\1222.PNG
working on 5\123.PNG
working on 5\124.PNG
working on 5\13.PNG
working on 5\14.PNG
working on 5\2.PNG
working on 5\21210307_64436331-fb8c-4866-9984-2eac675f4c93.jpg
working on 5\234.PNG
working on 5\244.PNG
working on 5\314.PNG
working on 5\345.PNG
working on 5\4.PNG
working on 5\43.PNG
working on 5\44.PNG
working on 5\444.PNG
working on 5\45.PNG
working on 5\450.PNG
working on 5\476.PNG
working on 5\498c7704c23500eedb732326a3c5cf57.jpg
working on 5\4e9228733f4b6757b5cd98ce688c2333.jpg
working on 5\5364769_2d3ba616-1ecf-4bc0-9b52-118b53ca2ec2.jpg
working on 5\55.PNG
working on 5\90.PNG
working on 5\901.PNG
working on 5\91.PNG
working on 5\99.PNG
working o

In [12]:
list_method = ["MI"]
for method in list_method:
    source_train = "C:\\SAMUEL\\KULIAH\\Skripsi Gasal 1920\\jupyter-notebook\\dataset_batik\\Training2\\"
    dest_train = f"C:\\SAMUEL\\KULIAH\\Skripsi Gasal 1920\\jupyter-notebook\\new\\{method}_dataset\\"
    
#     source_val = "C:\\SAMUEL\\KULIAH\\Skripsi Gasal 1920\\jupyter-notebook\\dataset_batik\\Training1\\"
#     dest_val = f"C:\\SAMUEL\\KULIAH\\Skripsi Gasal 1920\\jupyter-notebook\\new\\{method}\\Validation\\"
    
    create_dataset_singleaugm(method, source_train, dest_train)
#     create_dataset_singleaugm(method, source_val, dest_val)

working on 0\0531-cl.jpg
working on 0\batik_Grompol_motif_w.jpg
working on 0\ceplok-bintang.jpg
working on 0\ceplok-bligon1.jpg
working on 0\SC015.jpg
working on 0\SC050.jpg
working on 0\YC047.jpg
working on 0\YC086.jpg
working on 0\Yogya-Cap-Cabut Solo-Truntum-Sogan (1).jpg
working on 0\Yogya-Cap-Ceplok Kawung-Ceplok,Kawung-Sogan.jpg
working on 0\Yogya-Cap-Ceplok Kembang Jeruk-Truntum, Ceplok,Parang-Sogan.jpg
working on 0\Yogya-Cap-Ceplok Matahari-Ceplok, Parang,Matahari-Sogan - Copy.jpg
working on 0\Yogya-Cap-Ceplok Sisik Bintang-Parang,Bintang-Sogan.jpg
working on 0\Yogya-Cap-Ceplok-Ceplok-Sogan.jpg
working on 0\Yogya-Cap-Grompol kecil-Grompol-Sogan (1).jpg
working on 0\Yogya-Cap-Grompol-Grompol-Sogan.jpg
working on 0\Yogya-Cap-Jamblang--Sogan.jpg
working on 0\Yogya-Cap-Sawitan Pintu Retno-Truntum-Sogan.jpg
working on 0\Yogya-Cap-Sidodrajat Truntum-truntum-,Sogan (1).jpg
working on 0\Yogya-Cap-Sidomulyo--Sogan (1).jpg
working on 0\Yogya-Cap-Sidomulyo--Sogan (4).jpg
working on 0\Yogy

working on 6\lung-lungan-ungu-latar-coklat.jpg
working on 6\lungan.png
working on 6\lunglungan1.png
working on 6\lunglungan2.png
working on 6\product9a.jpg
working on 6\SU_2086_BK__1_.JPG


In [13]:
source_dir = "C:\\SAMUEL\\KULIAH\\Skripsi Gasal 1920\\jupyter-notebook\\dataset_batik\\Training1\\"
dest_dir = "C:\\SAMUEL\\KULIAH\\Skripsi Gasal 1920\\jupyter-notebook\\new\\RA\\Training\\"

source_dir2 = "C:\\SAMUEL\\KULIAH\\Skripsi Gasal 1920\\jupyter-notebook\\dataset_batik\\Testing1\\"
dest_dir2 = "C:\\SAMUEL\\KULIAH\\Skripsi Gasal 1920\\jupyter-notebook\\new\\RA\\Validation\\"

create_dataset_singleaugm("RA", source_dir, dest_dir)
create_dataset_singleaugm("RA", source_dir2, dest_dir2)

working on 0\0531-cl.jpg
working on 0\KP019.jpg
working on 0\SC015.jpg
working on 0\Truntum Mangkoro.jpg
working on 0\wiyar-solo-tulis-petilan-ceplok-bledak.jpg
working on 0\Yogya-Cap-Ceplok Sisik Bintang-Parang,Bintang-Sogan.jpg
working on 0\Yogya-Cap-Cepoko Mulyo--Sogan.jpg
working on 0\Yogya-Cap-Grompol kecil-Grompol-Sogan (1).jpg
working on 0\Yogya-Cap-Guntur Madu-Kawung,Parang-Sogan.jpg
working on 0\Yogya-Cap-Sawitan Pintu Retno-Truntum-Sogan.jpg
working on 0\Yogya-Cap-Tambal-Truntum,Parang,Kawung-Sogan (2).jpg
working on 0\Yogya-Tulis-Cantel Wajik Gurdo-Gurdo-Sogan.jpg
working on 0\Yogya-Tulis-Ceplok Ukel Kembang Oval-Truntum,Ukel-Sogan.jpg
working on 0\Yogya-Tulis-Keong Renteng-Truntum-Sogan.jpg
working on 0\Yogya-Tulis-Lengko-Truntum-Sogan.jpg
working on 0\Yogya-Tulis-Plonthang Kencono-Truntum-Sogan.jpg
working on 0\Yogya-Tulis-Prabu Anom Truntum-Truntum,Gurdo-Sogan.jpg
working on 0\Yogya-Tulis-Putri Solo Ukel-Truntum,Gurdo,Ukel-Sogan.jpg
working on 0\Yogya-Tulis-Suryoningrat-C

In [24]:
source_dir = "C:\\SAMUEL\\KULIAH\\Skripsi Gasal 1920\\jupyter-notebook\\dataset_batik\\Testing2\\"
dest_dir = "C:\\SAMUEL\\KULIAH\\Skripsi Gasal 1920\\jupyter-notebook\\new\\testing_dataset_gray\\"

# source_dir2 = "C:\\SAMUEL\\KULIAH\\Skripsi Gasal 1920\\jupyter-notebook\\dataset_batik\\Testing1\\"
# dest_dir2 = "C:\\SAMUEL\\KULIAH\\Skripsi Gasal 1920\\jupyter-notebook\\new\\Validation\\"

create_dataset_noaugm(source_dir, dest_dir)
# create_augmented_dataset_allfeatures_v2(source_dir, dest_dir)
# create_augmented_dataset_allfeatures(source_dir2, dest_dir2)

working on 0\bbr-004-batik-yogya-kombinasi-rakitan-putri-solo.jpg
working on 0\ceplok-ganggong.jpg
working on 0\Solo-Cap-Parang-Ceplok Teblem.jpg
working on 0\Yogya-Cap-Purbonegoro-Truntum-Sogan.jpg
working on 0\Yogya-Cap-Tambal-Truntum,Parang,Kawung-Sogan (2).jpg
working on 0\Yogya-Cap-Truntum-Truntum-Sogan (1).jpg
working on 0\Yogya-Tulis-Keong Renteng-Truntum-Sogan.jpg
working on 0\Yogya-Tulis-Langen Mulyo-Truntum-Sogan.jpg
working on 0\Yogya-Tulis-Putri Solo Ukel-Truntum,Gurdo,Ukel-Sogan.jpg
working on 0\Yogya-Tulis-Ukel Ceplok Bintang-Ukel,Bintang,Ceplok-Sogan.jpg
working on 1\10-nitik-motif-batik-kawung.jpg
working on 1\Batik_pattern_-_kawung.jpg
working on 1\YC004.jpg
working on 1\YC035.jpg
working on 1\YC127.jpg
working on 1\YK047.jpg
working on 1\YK048.jpg
working on 1\YT026.jpg
working on 1\YT084.jpg
working on 1\YT175.jpg
working on 2\0166-cl.jpg
working on 2\8agt 042.jpg
working on 2\dfasfaew.JPG
working on 2\IMG_2621u1.jpg
working on 2\lereng kembang.JPG
working on 2\Leren

In [16]:
# source_dir = "C:\\SAMUEL\\KULIAH\\Skripsi Gasal 1920\\jupyter-notebook\\dataset_batik\\Training1\\"
# dest_dir = "C:\\SAMUEL\\KULIAH\\Skripsi Gasal 1920\\jupyter-notebook\\new\\Training\\"

source_dir2 = "C:\\SAMUEL\\KULIAH\\Skripsi Gasal 1920\\jupyter-notebook\\dataset_batik\\Testing2\\"
dest_dir2 = "C:\\SAMUEL\\KULIAH\\Skripsi Gasal 1920\\jupyter-notebook\\new\\Validation\\"

# create_dataset_noaugm(source_dir, dest_dir)
create_dataset_noaugm(source_dir2, dest_dir2)

working on 0\bbr-004-batik-yogya-kombinasi-rakitan-putri-solo.jpg
working on 0\ceplok-ganggong.jpg
working on 0\Solo-Cap-Parang-Ceplok Teblem.jpg
working on 0\Yogya-Cap-Purbonegoro-Truntum-Sogan.jpg
working on 0\Yogya-Cap-Tambal-Truntum,Parang,Kawung-Sogan (2).jpg
working on 0\Yogya-Cap-Truntum-Truntum-Sogan (1).jpg
working on 0\Yogya-Tulis-Keong Renteng-Truntum-Sogan.jpg
working on 0\Yogya-Tulis-Langen Mulyo-Truntum-Sogan.jpg
working on 0\Yogya-Tulis-Putri Solo Ukel-Truntum,Gurdo,Ukel-Sogan.jpg
working on 0\Yogya-Tulis-Ukel Ceplok Bintang-Ukel,Bintang,Ceplok-Sogan.jpg
working on 1\10-nitik-motif-batik-kawung.jpg
working on 1\Batik_pattern_-_kawung.jpg
working on 1\YC004.jpg
working on 1\YC035.jpg
working on 1\YC127.jpg
working on 1\YK047.jpg
working on 1\YK048.jpg
working on 1\YT026.jpg
working on 1\YT084.jpg
working on 1\YT175.jpg
working on 2\0166-cl.jpg
working on 2\8agt 042.jpg
working on 2\dfasfaew.JPG
working on 2\IMG_2621u1.jpg
working on 2\lereng kembang.JPG
working on 2\Leren

In [12]:
# To create dataset for nongeometri batik.
# this dataset will be fed into GAN to generate synthetic images
sourcedir = "C:\\SAMUEL\\KULIAH\\Skripsi Gasal 1920\\jupyter-notebook\\dataset_batik\\nongeometri\\nongeometri_labeled\\"
destdir = "C:\\SAMUEL\\KULIAH\\Skripsi Gasal 1920\\jupyter-notebook\\dataset_batik\\nongeometri\\new\\"
create_augmented_nongeometri_for_GAN(sourcedir, destdir)

working on 5\10.PNG
working on 5\100.PNG
working on 5\102.PNG
working on 5\1023.PNG
working on 5\109.PNG
working on 5\11.PNG
working on 5\111.PNG
working on 5\122.PNG
working on 5\1222.PNG
working on 5\123.PNG
working on 5\124.PNG
working on 5\13.PNG
working on 5\14.PNG
working on 5\2.PNG
working on 5\234.PNG
working on 5\244.PNG
working on 5\314.PNG
working on 5\345.PNG
working on 5\4.PNG
working on 5\43.PNG
working on 5\44.PNG
working on 5\444.PNG
working on 5\45.PNG
working on 5\450.PNG
working on 5\476.PNG
working on 5\55.PNG
working on 5\90.PNG
working on 5\901.PNG
working on 5\91.PNG
working on 5\99.PNG
working on 5\999.PNG
working on 5\Capture.PNG
FOUND OTHER TYPE: Semen.rar
working on 5\Yogya-Cap-Sidomukti Cemeng-Gurdo-Sogan.jpg
working on 5\Yogya-Cap-Sidomukti Sringit--Sogan.jpg
working on 5\Yogya-Sidomukti Sringit--Sogan (1).jpg
working on 5\Yogya-Tulis-Ceplok Asih-Truntum,Gurdo-Sogan.jpg
working on 5\Yogya-Tulis-Sido mukti--Sogan.jpg
working on 5\Yogya-Tulis-Sidomukti--Sogan

In [22]:
source_train = "C:\\SAMUEL\\KULIAH\\Skripsi Gasal 1920\\jupyter-notebook\\dataset_batik\\Training2\\"
dest_train = f"C:\\SAMUEL\\KULIAH\\Skripsi Gasal 1920\\jupyter-notebook\\new\\Training_gray\\"

create_augmented_dataset_allfeatures(source_train, dest_train)

working on 0\0531-cl.jpg
working on 0\batik_Grompol_motif_w.jpg
working on 0\ceplok-bintang.jpg
working on 0\ceplok-bligon1.jpg
working on 0\SC015.jpg
working on 0\SC050.jpg
working on 0\YC047.jpg
working on 0\YC086.jpg
working on 0\Yogya-Cap-Cabut Solo-Truntum-Sogan (1).jpg
working on 0\Yogya-Cap-Ceplok Kawung-Ceplok,Kawung-Sogan.jpg
working on 0\Yogya-Cap-Ceplok Kembang Jeruk-Truntum, Ceplok,Parang-Sogan.jpg
working on 0\Yogya-Cap-Ceplok Matahari-Ceplok, Parang,Matahari-Sogan - Copy.jpg
working on 0\Yogya-Cap-Ceplok Sisik Bintang-Parang,Bintang-Sogan.jpg
working on 0\Yogya-Cap-Ceplok-Ceplok-Sogan.jpg
working on 0\Yogya-Cap-Grompol kecil-Grompol-Sogan (1).jpg
working on 0\Yogya-Cap-Grompol-Grompol-Sogan.jpg
working on 0\Yogya-Cap-Jamblang--Sogan.jpg
working on 0\Yogya-Cap-Sawitan Pintu Retno-Truntum-Sogan.jpg
working on 0\Yogya-Cap-Sidodrajat Truntum-truntum-,Sogan (1).jpg
working on 0\Yogya-Cap-Sidomulyo--Sogan (1).jpg
working on 0\Yogya-Cap-Sidomulyo--Sogan (4).jpg
working on 0\Yogy

working on 3\Yogya-Cap-Grompol kecil-Grompol-Sogan (1).jpg
working on 3\Yogya-Nitik--Sogan (21).jpg
working on 3\Yogya-Nitik-Cakar-Sogan.jpg
working on 3\Yogya-Nitik-Ceplok,Nitik-Sogan.jpg
working on 3\Yogya-Nitik-Nitik,Cakar-Sogan (1).jpg
working on 3\Yogya-Nitik-Nitik,Cakar-Sogan (3).jpg
working on 3\Yogya-Nitik-Nitik,Sekar-Sogan (10).jpg
working on 3\Yogya-Nitik-Nitik,Sekar-Sogan (11).jpg
working on 3\Yogya-Nitik-Nitik,Sekar-Sogan (12).jpg
working on 3\Yogya-Nitik-Nitik,Sekar-Sogan (2).jpg
working on 3\Yogya-Nitik-Nitik,Sekar-Sogan (3).jpg
working on 3\Yogya-Nitik-Nitik,Sekar-Sogan (4).jpg
working on 3\Yogya-Nitik-Nitik,Sekar-Sogan (7).jpg
working on 3\Yogya-Nitik-Nitik,Sekar-Sogan (8).jpg
working on 3\Yogya-Nitik-Nitik,Sekar-Sogan (9).jpg
working on 3\Yogya-Nitik-Nitik-Sogan (10).jpg
working on 3\Yogya-Nitik-Nitik-Sogan (17).jpg
working on 3\Yogya-Nitik-Nitik-Sogan (18).jpg
working on 3\Yogya-Nitik-Nitik-Sogan (19).jpg
working on 3\Yogya-Nitik-Nitik-Sogan (2).jpg
working on 3\Yogya

working on 6\lungan.png
working on 6\lunglungan1.png
working on 6\lunglungan2.png
working on 6\product9a.jpg
working on 6\SU_2086_BK__1_.JPG


### DOING RANDOM EXPERIMENT BELOW

In [5]:
shape = (3,3,3)
if shape > (250,250,):
    print("yes")
else:
    print("no")

no


In [9]:
str_path = "C:\\SAMUEL\\KULIAH\\Skripsi Gasal 1920\\jupyter-notebook\\dataset_batik\\Training\\0\\0531-cl.jpg"
img = imageio.imread(str_path)
np.shape(img)

if np.shape(img) > (448,448,):
    print("yes")
else:
    print("no")

no


In [57]:
img_others = imageio.imread(f"C:\\SAMUEL\\KULIAH\\Skripsi Gasal 1920\\jupyter-notebook\\dataset_batik\\Training\\0\\0531-cl.jpg")

3